In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression

import pandoc

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

import math

import statsmodels.formula.api as smf

from sklearn.metrics import roc_curve, roc_auc_score
from scipy.stats import chi2
from statsmodels.stats.diagnostic import linear_harvey_collier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.proportion import proportions_chisquare
from statsmodels.stats.api import het_breuschpagan

import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.outliers_influence import summary_table
from statsmodels.stats.diagnostic import linear_reset

from statsmodels.stats.diagnostic import lilliefors
from statsmodels.stats.diagnostic import het_white

from sklearn.metrics import roc_auc_score, roc_curve
from statsmodels.stats.diagnostic import het_breuschpagan

from statsmodels.stats.proportion import proportions_chisquare
from statsmodels.stats.api import het_white

from statsmodels.stats.diagnostic import linear_harvey_collier


from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.api import het_white

from statsmodels.stats.diagnostic import het_white
import os


In [2]:
stats_path = '/Users/caseymoser/Desktop/UFC Analysis/UFC/Go to Ground Analysis/ufc_fight_stats.csv'

results_path = '/Users/caseymoser/Desktop/UFC Analysis/UFC/Go to Ground Analysis/ufc_fight_results.csv'

stats_df = pd.read_csv(stats_path)

results_df = pd.read_csv(results_path)


In [4]:
# Hypothesis is that strikers with more of a grappling base (e.g. bjj blackbelt or wrestling) have a greater finish rate because they are more likely to go to the ground

stats_df

,EVENT,BOUT,ROUND,FIGHTER,KD,SIG.STR.,SIG.STR. %,TOTAL STR.,TD,TD %,SUB.ATT,REV.,CTRL,HEAD,BODY,LEG,DISTANCE,CLINCH,GROUND
0,UFC Fight Night: Taira vs. Park,Tatsuro Taira vs. HyunSung Park,Round 1,Tatsuro Taira,1.0,6 of 9,66%,12 of 18,2 of 2,100%,0.0,0.0,3:56,5 of 8,0 of 0,1 of 1,3 of 5,0 of 0,3 of 4
1,UFC Fight Night: Taira vs. Park,Tatsuro Taira vs. HyunSung Park,Round 2,Tatsuro Taira,0.0,1 of 2,50%,1 of 2,2 of 3,66%,1.0,0.0,0:45,1 of 2,0 of 0,0 of 0,1 of 2,0 of 0,0 of 0
2,UFC Fight Night: Taira vs. Park,Tatsuro Taira vs. HyunSung Park,Round 1,HyunSung Park,0.0,2 of 5,40%,14 of 17,0 of 0,---,0.0,0.0,0:00,1 of 3,0 of 0,1 of 2,1 of 3,0 of 1,1 of 1
3,UFC Fight Night: Taira vs. Park,Tatsuro Taira vs. HyunSung Park,Round 2,HyunSung Park,0.0,1 of 4,25%,1 of 4,0 of 0,---,0.0,0.0,0:00,0 of 3,0 of 0,1 of 1,1 of 4,0 of 0,0 of 0
4,UFC Fight Night: Taira vs. Park,Mateusz Rebecki vs. Chris Duncan,Round 1,Mateusz Rebecki,0.0,32 of 74,43%,38 of 81,0 of 1,0%,0.0,0.0,1:00,20 of 61,1 of 2,11 of 11,30 of 72,2 of 2,0 of 0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38785,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,Round 1,David Levicki,0.0,4 of 5,80%,95 of 102,0 of 0,---,0.0,0.0,--,4 of 5,0 of 0,0 of 0,1 of 2,2 of 2,1 of 1
38786,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Patrick Smith,0.0,1 of 1,100%,1 of 1,0 of 1,0%,1.0,0.0,--,0 of 0,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0
38787,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Ray Wizard,0.0,1 of 1,100%,2 of 2,0 of 0,---,0.0,0.0,--,0 of 0,0 of 0,1 of 1,1 of 1,0 of 0,0 of 0
38788,UFC 2: No Way Out,Scott Morris vs. Sean Daugherty,Round 1,Scott Morris,0.0,1 of 1,100%,2 of 2,1 of 1,100%,1.0,0.0,--,1 of 1,0 of 0,0 of 0,0 of 0,1 of 1,0 of 0


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

def get_ufc_fighters_list():
    url = "https://en.wikipedia.org/wiki/List_of_UFC_fighters"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    fighters = set()
    tables = soup.find_all('table', {'class': 'wikitable'})

    for table in tables:
        for link in table.find_all('a', href=True):
            name = link.get_text(strip=True)
            href = link['href']
            if href.startswith('/wiki/') and ':' not in href:
                fighters.add((name, 'https://en.wikipedia.org' + href))

    return list(fighters)

def extract_infobox(soup):
    """Extract key-value pairs from the infobox."""
    info = {}
    infobox = soup.find('table', {'class': 'infobox'})
    if not infobox:
        return info

    for row in infobox.find_all('tr'):
        header = row.find('th')
        data = row.find('td')
        if header and data:
            key = header.get_text(strip=True).lower()
            value = data.get_text(" ", strip=True).lower()
            info[key] = value
    return info

def parse_belt_rank(text, martial_art):
    """Try to find belt rank from text for BJJ or Judo."""
    if not text:
        return None
    # look for belts and ranks with regex
    belts = ['black belt', 'brown belt', 'purple belt', 'blue belt', 'white belt', 'red belt', 'master belt', 'dan']
    found = []
    for belt in belts:
        if belt in text:
            found.append(belt)
    # also look for dan ranks like "3rd dan"
    dan_match = re.findall(r'(\d+)(st|nd|rd|th)? dan', text)
    if dan_match:
        found.extend([f"{m[0]} dan" for m in dan_match])
    if found:
        return ", ".join(found)
    return None

def check_olympic(text, sport):
    if not text:
        return 'No'
    if 'olympic' in text and sport in text:
        return 'Yes'
    return 'No'

def check_master_of_sport(text):
    if not text:
        return 'No'
    if 'master of sport' in text:
        return 'Yes'
    return 'No'

def get_wrestling_bjj_info(fighter_name, url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract infobox data
        infobox_data = extract_infobox(soup)

        # Full text for keyword searching
        full_text = soup.get_text(separator=' ', strip=True).lower()

        # Columns initialization
        bjj_belt_rank = None
        judo_belt_rank = None
        master_of_sport = 'No'
        olympic_wrestler = 'No'
        olympic_judo = 'No'
        wrestling_background = []
        bjj_background = []
        other_styles = []

        # Check infobox keys for style or rank info
        # Common keys: style, martial art, rank, occupation, etc.
        style_text = ""
        for key in infobox_data:
            if 'style' in key or 'martial art' in key or 'rank' in key or 'occupation' in key:
                style_text += " " + infobox_data[key]

        bjj_belt_rank = parse_belt_rank(style_text, 'bjj')
        judo_belt_rank = parse_belt_rank(style_text, 'judo')

        master_of_sport = check_master_of_sport(style_text)
        olympic_wrestler = check_olympic(style_text, 'wrestling')
        olympic_judo = check_olympic(style_text, 'judo')

        # Extract sentences about wrestling and BJJ
        sentences = re.split(r'(?<=[.!?]) +', full_text)
        wrestling_keywords = ['wrestling', 'wrestler', 'ncaa wrestling', 'folkstyle wrestling', 'freestyle wrestling', 'wrestled']
        bjj_keywords = ['brazilian jiu-jitsu', 'bjj', 'black belt', 'brown belt', 'purple belt', 'blue belt', 'white belt']
        other_styles_keywords = ['boxing', 'muay thai', 'kickboxing', 'karate', 'taekwondo', 'sambo', 'judo']

        for sentence in sentences:
            lower_sent = sentence.lower()
            if any(word in lower_sent for word in wrestling_keywords):
                wrestling_background.append(sentence.strip())
            if any(word in lower_sent for word in bjj_keywords):
                bjj_background.append(sentence.strip())
            if any(word in lower_sent for word in other_styles_keywords):
                other_styles.append(sentence.strip())

        return {
            'fighter': fighter_name,
            'bjj_belt_rank': bjj_belt_rank,
            'judo_belt_rank': judo_belt_rank,
            'master_of_sport': master_of_sport,
            'olympic_wrestler': olympic_wrestler,
            'olympic_judo': olympic_judo,
            'wrestling_background': " | ".join(wrestling_background[:3]),
            'bjj_background': " | ".join(bjj_background[:3]),
            'other_styles_summary': " | ".join(other_styles[:3]),
            'wiki_url': url
        }

    except Exception as e:
        print(f"Error scraping {fighter_name}: {e}")
        return None

def main():
    fighters = get_ufc_fighters_list()
    print(f"Found {len(fighters)} fighters.")

    data = []
    count = 0
    for name, url in fighters:
        print(f"Processing: {name}")
        info = get_wrestling_bjj_info(name, url)
        if info:
            data.append(info)
        count += 1
        time.sleep(1)  # polite delay

        # For demo purposes, limit to 20 fighters (remove or increase for full scrape)
        if count >= 20:
            break

    df = pd.DataFrame(data)
    print(df.head())

    # Save to CSV for easy use
    df.to_csv("ufc_fighters_martial_arts_background.csv", index=False)
    print("Saved to ufc_fighters_martial_arts_background.csv")

if __name__ == "__main__":
    main()
